# Extract SAP VIEW | SCHEMA | COLUMN NAME | DATA TYPE


In [0]:

from pyspark.sql.functions import split, col, trim, countDistinct, collect_set, lower, when, regexp_replace, array_contains, explode, collect_list, array, explode_outer, upper
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Get mapping table information

In [0]:
query_map= '''
select * from hive_metastore.codeconverter_config.mapping_sap_dbx;
'''
map_df = spark.sql(query_map)
map_df = map_df.withColumn("CatalogName", split(col("SLO_Object"), r"\{ENV\}")[0])
map_df = map_df.withColumn("SchemaName", split(col("SLO_Object"), r"\.")[1])
map_df = map_df.withColumn("TableName", split(col("SLO_Object"), r"\.")[2])

source_package = map_df.select('source_package').distinct().collect()[0]['source_package']


sap_databricks_lst_tup = []

rows = map_df.collect()

for row in rows:
    table_id = row['TableName']
    decoupling_view = row['Decoupling_View']
    sap_databricks_lst_tup.append((table_id, decoupling_view))


lst_of_cvs = [tup[1] for tup in sap_databricks_lst_tup]
list_of_tab = [tup[0] for tup in sap_databricks_lst_tup]


display(map_df)

# Get SAP Object Info

In [0]:

copy_data_flag = dbutils.widgets.get("copy_data_from_sap")

if copy_data_flag == "YES":
    
    sap_hana_host = "vbh1db.sap.insim.biz"
    sap_hana_port = "30615"
    sap_hana_user = dbutils.secrets.get(scope="BDT mall", key="SAP-HANA-username")
    sap_hana_password = dbutils.secrets.get(scope="BDT mall", key="SAP-HANA-password")
    sap_hana_jdbc_url = f"jdbc:sap://{sap_hana_host}:{sap_hana_port}?encrypt=true&validateCertificate=false"

    #dataflow = source_package + '%'
    dataflow = 'NNROOT.Z00_Applications.LEGO.UPO_MPO%'


    query_text_1 = f"""
        SELECT DISTINCT VIEW_NAME, SCHEMA_NAME
        FROM SYS.VIEWS
        WHERE VIEW_NAME LIKE '{dataflow}'
        AND VIEW_TYPE = 'CALC'
        ORDER BY VIEW_NAME
    """

    # Execute the first query
    df_views = spark.read.format("jdbc") \
        .option("Driver", "com.sap.db.jdbc.Driver") \
        .option("url", sap_hana_jdbc_url) \
        .option("user", sap_hana_user) \
        .option("password", sap_hana_password) \
        .option("query", query_text_1) \
        .load()

    # Collect the results
    view_names = df_views.select("VIEW_NAME", "SCHEMA_NAME").distinct().collect()
    display(view_names)

    schema = StructType([StructField("SCHEMA_NAME", StringType(), True), StructField("VIEW_NAME", StringType(), True), StructField("COLUMN_NAME", StringType(), True), 
                        StructField("DATA_TYPE_ID", IntegerType(), True), StructField("DATA_TYPE_NAME", StringType(), True)
    ])

    # Create an empty DataFrame to store the results
    final_df = spark.createDataFrame([], schema=schema)

    # Loop through the collected results and execute the second query for each pair
    for row in view_names:
        view_name = row["VIEW_NAME"]
        schema_name = row["SCHEMA_NAME"]

        # Define the second query
        query_text_2 = f"""
            SELECT SCHEMA_NAME, VIEW_NAME, COLUMN_NAME, DATA_TYPE_ID, DATA_TYPE_NAME
            FROM SYS.VIEW_COLUMNS
            WHERE SCHEMA_NAME = '{schema_name}'
            AND VIEW_NAME = '{view_name}'
        """

        # Execute the second query
        temp_df = spark.read.format("jdbc") \
            .option("Driver", "com.sap.db.jdbc.Driver") \
            .option("url", sap_hana_jdbc_url) \
            .option("user", sap_hana_user) \
            .option("password", sap_hana_password) \
            .option("query", query_text_2) \
            .load()

        # # Add the VIEW_NAME and SCHEMA_NAME columns to the temp_df
        # temp_df = temp_df.withColumn("VIEW_NAME", lit(view_name))
        # temp_df = temp_df.withColumn("SCHEMA_NAME", lit(schema_name))

        # Append the results to the final DataFrame
        final_df = final_df.union(temp_df)

        #dbutils.widgets.put("copy_data_from_sap", "NO")

    final_df.write.mode("overwrite").saveAsTable("hive_metastore.codeconverter_config.sap_retention_datatype")





# Store SAP output into dbx table

In [0]:
# Get sap info stored in dbx
final_df = spark.sql("""SELECT * FROM hive_metastore.codeconverter_config.sap_retention_datatype""") 

In [0]:

final_df = final_df.withColumn("CalculationViewName", trim(split(col("VIEW_NAME"),"/")[1]))

# Filter df on target calculation views
final_df = final_df.filter(col("CalculationViewName").isin(lst_of_cvs))

no_of_unique_views = final_df.select("CalculationViewName").distinct().count()
print(f"Number of unique calculation views: {no_of_unique_views}")


print(f"Number of rows in initial SAP results: {final_df.count()}")
display(final_df)


In [0]:

# Filter for unique combinations of data_type and column_name
unique_df = final_df.dropDuplicates([ "CalculationViewName", "COLUMN_NAME", "DATA_TYPE_NAME"])

# Select only COLUMN_NAME, DATA_TYPE_NAME, and CalculationViewName
initial_df_filtered = unique_df.select("COLUMN_NAME", "DATA_TYPE_NAME", "CalculationViewName")



# Data from mapping table for filtering purpose 
sap_metadata_filter = map_df.select(['TableName', 'Decoupling_View'])
sap_metadata_filter = sap_metadata_filter.dropDuplicates(["TableName", "Decoupling_View"])
sap_metadata_filter = sap_metadata_filter.filter(col("Table_ID") != '')


filtered_df = initial_df_filtered.join(sap_metadata_filter, initial_df_filtered.CalculationViewName == sap_metadata_filter.Decoupling_View, "inner")


# Group by COLUMN_NAME, count distinct DATA_TYPE_NAMEs, and collect DATA_TYPE_NAMEs and CalculationViewNames
grouped_df_count = filtered_df.groupBy(["CalculationViewName", "TableName", "COLUMN_NAME", "DATA_TYPE_NAME"]).count()

grouped_df =  grouped_df_count.select(["CalculationViewName", "TableName","COLUMN_NAME", "DATA_TYPE_NAME"])


display(grouped_df)


# EXTRACT DATABRICKS TABLE AND VIEW INFO 

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import lit, concat

# Initialize an empty list to collect rows for DataFrame creation
metadata_rows = []
# Function to describe an object and collect its metadata
def describe_object(object_type, catalog_name, schema_name, object_name):
    describe_df = spark.sql(f"DESCRIBE {catalog_name}.{schema_name}.{object_name}")
    for row in describe_df.collect():
        column_name = row["col_name"].upper()
        data_type = row["data_type"].upper()
        metadata_rows.append(Row(object_type=object_type, catalog_name=catalog_name, schema_name=schema_name, object_name=object_name, column_name=column_name, data_type=data_type))


# Filter out rows where CatalogName is empty or starts with "Manual" or "Stable"
filtered_map_df = map_df.filter(
    (col("CatalogName") != '') &
    (~col("CatalogName").startswith("Manual")) &
    (~col("CatalogName").startswith("Stable"))
    )

# Add the suffix "devdev" to the CatalogName column
filtered_map_df = filtered_map_df.withColumn("CatalogName", concat(col("CatalogName"), lit("devdev")))


# Collect unique catalog and schema combinations
catalog_schema_pairs = filtered_map_df.select("CatalogName", "SchemaName").distinct().collect()


for pair in catalog_schema_pairs:
    catalog_name = pair["CatalogName"]
    schema_name = pair["SchemaName"]

    # Process tables
    dbx_tables = spark.sql(f"SHOW TABLES IN {catalog_name}.{schema_name}")
    table_names = list(set([row["tableName"].upper() for row in dbx_tables.collect()]))  # Remove duplicates

    for table in table_names:
        describe_object('TABLE', catalog_name, schema_name, table)

    # Process views
    spark.sql(f"USE CATALOG {catalog_name}")
    dbx_views = spark.sql(f"SHOW VIEWS IN {schema_name}")
    view_names = list(set([row["viewName"].upper() for row in dbx_views.collect()]))  # Remove duplicates

    for view in view_names:
        if view in table_names:
            # Compare structures
            table_structure = spark.sql(f"DESCRIBE {catalog_name}.{schema_name}.{view}").collect()
            view_structure = spark.sql(f"DESCRIBE {schema_name}.{view}").collect()
            if table_structure != view_structure:
                describe_object('VIEW', catalog_name, schema_name, view)
        else:
            describe_object('VIEW', catalog_name, schema_name, view)

# Create a DataFrame from the list of rows
dbx_metadata_df = spark.createDataFrame(metadata_rows)

# Show the DataFrame to inspect the results
display(dbx_metadata_df)


In [0]:

# Filter to only one column, remove null records, and convert to uppercase
scope_tables_df = map_df.select("TableName").filter(col("TableName").isNotNull()).withColumn("TableName", upper(col("TableName")))

# Join the Databricks and SAP metadata DataFrames
joined_df = dbx_metadata_df.join(scope_tables_df, dbx_metadata_df.object_name == scope_tables_df.TableName, "inner")

# Remove redundant columns 
joined_df = joined_df.drop("TableName")

# Check number of unique tables in joined_df
print(f"Number of unique tables in joined_df: {joined_df.select('object_name').distinct().count()}")

# Display final dataframe 
display(joined_df)

# COMPARE SAP AND DATABRICKS DATATYPES 

In [0]:
from pyspark.sql.functions import col, upper, concat, lit, regexp_replace, when, coalesce

# Select and rename columns from SAP metadata
sap_metadata = grouped_df.select(
    col("CalculationViewName").alias("CalcViewName"),
    upper(col("TableName")).alias("TableName"),
    col("COLUMN_NAME").alias("ColumnNameSAP"),
    col("DATA_TYPE_NAME").alias("DataTypeSAP")
)

# Add the suffix "devdev" to the CatalogName column and select relevant columns from Databricks metadata
dbx_metadata = joined_df.withColumn("CatalogSchema", concat(col("catalog_name"), lit("."), col("schema_name")))
dbx_metadata = dbx_metadata.select(
    col("object_name").alias("TableNameDbx"),
    col("column_name").alias("ColumnNameDbx"),
    col("data_type").alias("DataTypeDbx"),
    col("CatalogSchema").alias("CatalogSchema")
)

# Perform a full outer join on both TableName and ColumnName
merged_df = sap_metadata.join(
    dbx_metadata,
    (sap_metadata.TableName == dbx_metadata.TableNameDbx) & (sap_metadata.ColumnNameSAP == dbx_metadata.ColumnNameDbx),
    "full_outer"
)

# Remove precision from the Databricks data type (e.g., "DECIMAL(10,2)" becomes "DECIMAL")
merged_df = merged_df.withColumn(
    "DataTypeDbx",
    regexp_replace(col("DataTypeDbx").cast("string"), r'\(.*\)', '')
)

# Fill null values with TableNameDbx, CalcViewName, and CatalogSchema
merged_df = merged_df.withColumn(
    "TableNameDbx",
    coalesce(col("TableNameDbx"), col("TableName"))
).withColumn(
    "CalcViewName",
    coalesce(col("CalcViewName"), col("CalcViewName"))
).withColumn(
    "CatalogSchema",
    coalesce(col("CatalogSchema"), col("CatalogSchema"))
)

# Determine if the data types match
merged_df = merged_df.withColumn(
    "DataTypeMatch",
    when(
        (col("DataTypeDbx").isNull() | col("DataTypeSAP").isNull()), 
        None
    ).when(
        (col("DataTypeDbx") == 'STRING') & (col('DataTypeSAP') == 'NVARCHAR'), 
        True
    ).otherwise(col("DataTypeDbx") == col("DataTypeSAP"))
)

# Select the final columns to display, including all columns from both sources
final_df = merged_df.select(
    'CatalogSchema', 
    'TableNameDbx', 
    'CalcViewName', 
    'ColumnNameDbx', 
    'ColumnNameSAP', 
    'DataTypeDbx', 
    'DataTypeSAP', 
    'DataTypeMatch'
)

# Display the final DataFrame
display(final_df)


In [0]:
from pyspark.sql.functions import (  
    col, upper, concat, lit, regexp_replace, when, coalesce  
)  
from pyspark.sql.types import StringType  
from pyspark.sql import functions as F  
  
# --- 1. Prepare SAP Metadata ---  
sap_metadata = grouped_df.select(  
    col("CalculationViewName").alias("CalcViewName"),  
    upper(col("TableName")).alias("TableName"),  
    col("COLUMN_NAME").alias("ColumnNameSAP"),  
    col("DATA_TYPE_NAME").alias("DataTypeSAP")  
)  
  
# --- 2. Prepare Databricks Metadata ---  
dbx_metadata = joined_df.withColumn(  
    "CatalogSchema", concat(col("catalog_name"), lit("."), col("schema_name"))  
).select(  
    col("object_name").alias("TableNameDbx"),  
    col("column_name").alias("ColumnNameDbx"),  
    col("data_type").alias("DataTypeDbx"),  
    col("CatalogSchema").alias("CatalogSchema")  
)  
  
# --- 3. Full Outer Join on Table Name and Column Name ---  
merged_df = sap_metadata.join(  
    dbx_metadata,  
    (sap_metadata.TableName == dbx_metadata.TableNameDbx) &   
    (sap_metadata.ColumnNameSAP == dbx_metadata.ColumnNameDbx),  
    "full_outer"  
)  
  
# --- 4. Remove precision from Databricks data type (e.g., DECIMAL(10,2) -> DECIMAL) ---  
merged_df = merged_df.withColumn(  
    "DataTypeDbx",  
    regexp_replace(col("DataTypeDbx").cast("string"), r'\(.*\)', '')  
)  
  
# --- 5. Fill null values with values from the other source ---  
merged_df = merged_df.withColumn(  
    "TableNameDbx",  
    coalesce(col("TableNameDbx"), col("TableName"))  
).withColumn(  
    "CalcViewName",  
    coalesce(col("CalcViewName"), col("CalcViewName"))  # No change; could perhaps drop  
).withColumn(  
    "CatalogSchema",  
    coalesce(col("CatalogSchema"), col("CatalogSchema"))  # No change; could perhaps drop  
)  
  
# --- 6. Determine if the data types match ---  
merged_df = merged_df.withColumn(  
    "DataTypeMatch",  
    when(  
        (col("DataTypeDbx").isNull() | col("DataTypeSAP").isNull()), None  
    ).when(  
        (col("DataTypeDbx") == 'STRING') & (col('DataTypeSAP') == 'NVARCHAR'), True  
    ).otherwise(col("DataTypeDbx") == col("DataTypeSAP"))  
)  
  
# --- 7. Select final columns (for clarity and next processing) ---  
final_df = merged_df.select(  
    'CatalogSchema',  
    'TableNameDbx',  
    'CalcViewName',  
    'ColumnNameDbx',  
    'ColumnNameSAP',  
    'DataTypeDbx',  
    'DataTypeSAP',  
    'DataTypeMatch'  
)  
  
# --- 8. Collect all unique non-null ColumnNameDbx values as a list on the driver ---  
col_names_dbx = [  
    row['ColumnNameDbx']  
    for row in final_df.select('ColumnNameDbx').distinct().na.drop().collect()  
]  
  
# --- 9. Define a UDF to find the first match (substring search) ---  
def find_matching_dbx(sap_col):  
    if sap_col is None:  
        return None  
    for dbx in col_names_dbx:  
        if sap_col in dbx:  
            return dbx  #
    return None
  
find_matching_dbx_udf = F.udf(find_matching_dbx, StringType())

final_df =  final_df.withColumn("MatchedCOlumnDbx", 
                                    when(
                                        col("ColumnNameDbx").isNull(), 
                                        find_matching_dbx_udf(col("ColumnNameSAP"))
                                    ).otherwise(None)
                                    )
display(final_df)

In [0]:
from pyspark.sql.types import StringType  


# Step 1: Clean data type and create lookup dict  
dbx_metadata_clean = dbx_metadata.withColumn(  
    "DataTypeDbx",  
    regexp_replace(col("DataTypeDbx").cast("string"), r'\(.*\)', '')  
)  
dbx_coltype_dict = {  
    (row.TableNameDbx, row.ColumnNameDbx): row.DataTypeDbx  
    for row in dbx_metadata_clean.select("TableNameDbx", "ColumnNameDbx", "DataTypeDbx").distinct().collect()  
}  
  
# Step 2: UDF for lookup  
def lookup_matched_datatype(tablename, matched_col):  
    if not tablename or not matched_col:  
        return None  
    return dbx_coltype_dict.get((tablename, matched_col), None)  
  
  
lookup_matched_datatype_udf = F.udf(lookup_matched_datatype, StringType())  
  
# Step 3: Add the new column 'DataTypeDbxMatched' for the matched Databricks column's data type  
final_df = final_df.withColumn(  
    "DataTypeDbxMatched",  
    lookup_matched_datatype_udf(  
        col("TableNameDbx"),  
        col("MatchedCOlumnDbx")   # Note: ensure your capitalization matches your actual column name!  
    )  
)  
  
# Step 4: Compare with SAP data type, store the result in 'DataTypeMatchAfterMatch'  
final_df = final_df.withColumn(  
    "DataTypeMatchAfterMatch",  
    when(  
        col("DataTypeDbxMatched").isNull() | col("DataTypeSAP").isNull(), None  
    ).when(  
        (col("DataTypeDbxMatched") == 'STRING') & (col('DataTypeSAP') == 'NVARCHAR'), True  
    ).otherwise(col("DataTypeDbxMatched") == col("DataTypeSAP"))  
)  
  
# Step 5: Show the results (or write for further use)  
final_df.select(  
    'CatalogSchema',  
    'TableNameDbx',  
    'CalcViewName',  
    'ColumnNameDbx',  
    'ColumnNameSAP',  
    'MatchedCOlumnDbx',  
    'DataTypeSAP',  
    'DataTypeDbx',  
    'DataTypeDbxMatched',  
    'DataTypeMatchAfterMatch'  
)

display(final_df)
